In [1]:
from channel import Channel
from field import FieldElement
from merkle import MerkleTree
from polynomial import interpolate_poly, Polynomial

In [2]:
# Trace
t = [FieldElement(1), FieldElement(3141592)]
while len(t) < 1023:
    t.append(t[-2] * t[-2] + t[-1] * t[-1])
# Mult group
g = FieldElement.generator() ** (3 * 2 ** 20)
points = [g ** i for i in range(1024)]
# Coset subgroup
h_gen = FieldElement.generator() ** ((2 ** 30 * 3) // 8192)
H = [h_gen ** i for i in range(8192)]
# domain
domain = [FieldElement.generator() * x for x in H]
# Interpolating polynomial
p = interpolate_poly(points[:-1], t)
# Domain eval
ev = [p.eval(d) for d in domain]
# domain eval merkle tree
mt = MerkleTree(ev)
ch = Channel()


100%|██████████| 1023/1023 [00:11<00:00, 88.97it/s]


In [3]:
numer0 = p - Polynomial([FieldElement(1)])
denom0 = Polynomial.gen_linear_term(FieldElement(1))
q0, r0 = numer0.qdiv(denom0)
numer1 = p - Polynomial([FieldElement(2338775057)])
denom1 = Polynomial.gen_linear_term(points[1022])
q1, r1 = numer1.qdiv(denom1)
inner_poly0 = Polynomial([FieldElement(0), points[2]])
final0 = p.compose(inner_poly0)
inner_poly1 = Polynomial([FieldElement(0), points[1]])
composition = p.compose(inner_poly1)
final1 = composition * composition
final2 = p * p
numer2 = final0 - final1 - final2
coef = [FieldElement(1)] + [FieldElement(0)] * 1023 + [FieldElement(-1)]
numerator_of_denom2 = Polynomial(coef).scalar_mul(-1)
factor0 = Polynomial.gen_linear_term(points[1021])
factor1 = Polynomial.gen_linear_term(points[1022])
factor2 = Polynomial.gen_linear_term(points[1023])
denom_of_denom2 = factor0 * factor1 * factor2
denom2, r_denom2 = numerator_of_denom2.qdiv(denom_of_denom2)
q2, r2 = numer2.qdiv(denom2)

In [4]:
# Prover provides linear factor to compose polynomial CP
alpha0 = ch.receive_random_field_element()
alpha1 = ch.receive_random_field_element()
alpha2 = ch.receive_random_field_element()
cp0 = q0.scalar_mul(alpha0)
cp1 = q1.scalar_mul(alpha1)
cp2 = q2.scalar_mul(alpha2)
cp = cp0 + cp1 + cp2
cp_ev = [cp.eval(d) for d in domain]
cp_mt = MerkleTree(cp_ev)

In [5]:
def next_fri_domain(domain):
    return [x ** 2 for x in domain[:len(domain) // 2]]


def next_fri_polynomial(poly, alpha):
    odd_coefficients = poly.poly[1::2]
    even_coefficients = poly.poly[::2]
    odd = Polynomial(odd_coefficients).scalar_mul(alpha)
    even = Polynomial(even_coefficients)
    return odd + even


def next_fri_layer(poly, dom, alpha):
    next_poly = next_fri_polynomial(poly, alpha)
    next_dom = next_fri_domain(dom)
    next_layer = [next_poly.eval(x) for x in next_dom]
    return next_poly, next_dom, next_layer

In [6]:
fri_polys = [cp]
fri_doms = [domain]
fri_layers = [cp_ev]
fri_mt = [cp_mt]
beta = []
while fri_polys[-1].degree() > 0:
    beta.append(ch.receive_random_field_element())
    next_poly, next_dom, next_layer = next_fri_layer(fri_polys[-1], fri_doms[-1], beta[-1])
    fri_polys.append(next_poly)
    fri_doms.append(next_dom)
    fri_layers.append(next_layer)
    fri_mt.append(MerkleTree(next_layer))
last_one = fri_polys[-1].poly[0] # poly 0 shall be constant

In [7]:
def decommit_on_fri_layers(idx):
    assert len(fri_layers) == len(fri_mt), f'layers size should be same as merkles size'
    res = []
    i = 0
    for layer, merkle in zip(fri_layers[:-1], fri_mt[:-1]):
        length = len(layer)
        idx = idx % length
        sib_idx = (idx + length // 2) % length
        assert len(layer) == len(fri_doms[i])
        assert layer[idx] == fri_polys[i](fri_doms[i][idx])
        assert layer[sib_idx] == fri_polys[i](-fri_doms[i][idx])
        res.append(layer[idx])
        res.append(merkle.get_authentication_path(idx))
        res.append(layer[sib_idx])
        res.append(merkle.get_authentication_path(sib_idx))
        i = i+1
    res.append(fri_layers[-1][0])
    return res

In [8]:
def decommit_on_query(idx):
    f_eval = ev
    f_merkle = mt
    assert idx + 16 < len(f_eval), f'query index: {idx} is out of range. Length of layer: {len(f_eval)}.'
    res = []
    res.append(f_eval[idx]) # f(x).
    res.append(f_merkle.get_authentication_path(idx)) # auth path for f(x).
    res.append(f_eval[idx + 8]) # f(gx).
    res.append(f_merkle.get_authentication_path(idx + 8)) # auth path for f(gx).
    res.append(f_eval[idx + 16]) # f(g^2x).
    res.append(f_merkle.get_authentication_path(idx + 16)) # auth path for f(g^2x).
    return res

In [9]:
# Prover knows
# * f eval on domain commitment, mt
# * cp eval on domain commitment, cp_mt
# * FRI protocol eval on each layer commiment, fri_mt
# * FRI last one commitment, last_one
# * size of domain, length
# * challenge alpha's for composing CP
# * challenge beta's on each layer of FRI operator
# * domain

In [10]:
# for single query
idx = ch.receive_random_int(0, 8191-16)
proof_f = decommit_on_query(idx)
proof_cp = decommit_on_fri_layers(idx)

In [11]:
# Basic check
assert len(proof_cp) % 2
v_last_one = proof_cp.pop()
assert v_last_one == last_one

In [12]:
from merkle import verify_decommitment

In [13]:
# Check polynomial constraint
# First check evaluation of polynomial f is honest
v_f_eval = proof_f[0::2]
v_f_auth = proof_f[1::2]
assert len(v_f_eval) == len(v_f_auth)
for i in range(len(v_f_eval)):
    assert verify_decommitment(idx + 8 * i, v_f_eval[i], v_f_auth[i], mt.root ), f'in iter {i}, go wrong'
# Then check whether they could satify the recursive condition
v_cp0 = proof_cp[:2]
length = len(fri_layers[0]) # 8192 shoud be a prior knowledge
assert verify_decommitment(idx % length, v_cp0[0], v_cp0[1], cp_mt.root)

In [14]:
# Check the polynomial constraint
fx = v_f_eval[0]
fgx = v_f_eval[1]
fggx = v_f_eval[2]
x = domain[idx]
p0 = (fx - 1) / (x - 1)
p1 = (fx - 2338775057) / (x - points[1022])
p2 = (fggx - fgx**2 - fx**2) / ((x**1024 - 1) / ((x - points[1021]) * (x - points[1022]) * (x - points[1023])))
assert v_cp0[0] == (alpha0 * p0 + alpha1 * p1 + alpha2 * p2)
print("poly constraint success!")

poly constraint success!


In [15]:
# Check low degree
v_cp = proof_cp[::2]
v_auth = proof_cp[1::2]
assert len(v_cp) == len(v_auth)
assert len(v_cp) % 2 == 0
k = length
for i in range(len(v_cp)//2):
    iter_idx = idx % k
    iter_sib_idx = (idx + k // 2) % k
    assert verify_decommitment(iter_idx, v_cp[2*i], v_auth[2*i], fri_mt[i].root)
    assert verify_decommitment(iter_sib_idx, v_cp[2*i+1], v_auth[2*i+1], fri_mt[i].root)
    k = k >> 1

In [16]:
k = length
x = domain[idx]
assert len(beta) == len(v_cp) // 2
for i in range(len(v_cp)//2 - 1):
    op1 = (v_cp[2*i] + v_cp[2*i + 1]) / FieldElement(2)
    op2 = (v_cp[2*i] - v_cp[2*i + 1]) / (FieldElement(2)*x)
    rhs = op1 + beta[i] * op2
    assert v_cp[2*(i+1)] == rhs, f" round {i}, CP(i+1) is {v_cp[2*(i+1)]} while rhs is {rhs}"
    x = x**2
print("low degree test success!")

low degree test success!


In [17]:
numerator_of_denom2 = Polynomial(coef)

In [20]:
numerator_of_denom2